## 1. Tải dữ liệu vietlott power 6/55 tự động theo draw ID chưa được tải

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# Tạo list những draw_ID đã tải về table vietlott.power.draw_result_power
cursor.execute('SELECT draw_ID FROM vietlott.power.draw_result_power ORDER BY draw_ID')
rows_draw = cursor.fetchall()
draw_result = [row[0] for row in rows_draw]

drawID_draw_result = [str(i).zfill(5) for i in draw_result]

# Tạo list những draw_ID đã tải về table vietlott.power.prize_result_power
cursor.execute('SELECT DISTINCT draw_ID FROM vietlott.power.prize_result_power ORDER BY draw_ID')
rows_prize = cursor.fetchall()
prize_result = [row[0] for row in rows_prize]

drawID_prize_result = [str(i).zfill(5) for i in prize_result]

# Tạo list chứa những draw_ID đã có từ trước đến hiện tại
winning_url = 'https://vietlott.vn/vi/trung-thuong/ket-qua-trung-thuong/winning-number-655'
response = requests.get(winning_url)
soup = BeautifulSoup(response.text, 'html.parser')
a_tag = soup.find('tbody').find('a')
max_draw_ID = a_tag.text.strip()
max_draw_ID = int(max_draw_ID)

list_drawID = [str(i).zfill(5) for i in range(1, max_draw_ID + 1)]

# Tạo list các drawID còn thiếu và chưa được tải về
missing_drawID = sorted((set(list_drawID) - set(drawID_draw_result)) | (set(list_drawID) - set(drawID_prize_result)))



if not missing_drawID:
    print('Đã có đầy đủ dữ liệu power 6/55, không cần crawl mới')
else:
    for drawID in missing_drawID:
        # URL của trang web cần crawl
        url = f'https://vietlott.vn/vi/trung-thuong/ket-qua-trung-thuong/655?id={drawID}&nocatche=1'


        # Gửi yêu cầu HTTP GET đến URL
        response = requests.get(url)

        # Sử dụng BeautifulSoup để phân tích cú pháp HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy giá trị ngày của dữ liệu
        h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
        date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
        date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

        ######### 1. Tải dữ liệu về số trúng giải #################

        # Tìm kiếm dữ liệu cần thiết
        # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
        draw_data = soup.find_all('div', class_='day_so_ket_qua_v2')

        # Lấy các kết quả trả về dưới dạng text
        for item in draw_data:
            draw = item.text
        # result = [item.text for item in draw_data]
        # Tách chuỗi thành số quả bóng và số đặc biệt
        balls, ball_special = draw.split('|')
        # Tách các số quả bóng
        ball_number = [balls[i:i+2] for i in range(1, len(balls), 2)]
        # Tạo dictionary
        draw_result = {'draw_ID': drawID, 'draw_date': date_object}
        draw_result.update({f'ball_{i+1}': ball_number[i] for i in range(len(ball_number))})
        draw_result['ball_special'] = ball_special
        # Tạo DataFrame từ dữ liệu và thêm vào danh sách
        draw_result = {key: [value] for key, value in draw_result.items()}
        df_draw_result = pd.DataFrame(draw_result)
        
        # Câu lệnh SQL để chèn dữ liệu vào table power.draw_result_power
        draw_result_query = '''
        MERGE INTO power.draw_result_power AS target
        USING (SELECT	  ? AS draw_ID
                        , ? AS draw_date
                        , ? AS ball_1
                        , ? AS ball_2
                        , ? AS ball_3
                        , ? AS ball_4
                        , ? AS ball_5
                        , ? AS ball_6
                        , ? AS ball_special
        ) AS source
        ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date
        WHEN MATCHED THEN
            UPDATE SET	target.ball_1 = source.ball_1, target.ball_2 = source.ball_2,
                        target.ball_3 = source.ball_3, target.ball_4 = source.ball_4,
                        target.ball_5 = source.ball_5, target.ball_6 = source.ball_6,
                        target.ball_special = source.ball_special
        WHEN NOT MATCHED BY TARGET THEN
            INSERT (draw_ID, draw_date, ball_1, ball_2, ball_3, ball_4, ball_5, ball_6, ball_special)
            VALUES (source.draw_ID, source.draw_date, source.ball_1, source.ball_2
                    , source.ball_3, source.ball_4, source.ball_5, source.ball_6, source.ball_special);
        '''
        # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
        for index, row in df_draw_result.iterrows():
            cursor.execute(draw_result_query, row['draw_ID'], row['draw_date'], row['ball_1'], row['ball_2']
                                            , row['ball_3'], row['ball_4'], row['ball_5'], row['ball_6'], row['ball_special'])


        ############################################

        ######### 2. Tải dữ liệu giá trị và số lượng giải #################
        # Tìm kiếm dữ liệu cần thiết
        # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
        prize_data = soup.find('div', class_='table-responsive').find('table')
        # Trích xuất tiêu đề cột
        headers = []
        for th in prize_data.find('thead').find_all('th'):
            headers.append(th.text.strip())
        # Trích xuất dữ liệu hàng
        rows = []
        for tr in prize_data.find('tbody').find_all('tr'):
            cells = tr.find_all('td')
            row_data = [cell.text.strip() for cell in cells]
            rows.append(row_data)
        # Tạo DataFrame
        df_prize_result = pd.DataFrame(rows, columns=headers)
        # Bỏ cột "Kết quả"
        df_prize_result = df_prize_result.drop(columns=["Kết quả"])
        # Đổi tên các cột
        df_prize_result = df_prize_result.rename(columns={
            "Giải thưởng": "prize",
            "Số lượng giải": "number_of_prizes",
            "Giá trị giải (đồng)": "prize_value"
        })
        # Chuyển đổi định dạng của cột number_of_prizes và prize_value
        # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
        df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
        df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')
        df_prize_result['draw_ID'] = drawID
        df_prize_result['draw_date'] = date_object
        # Thay đổi các giá trị cột 'prize'
        df_prize_result['prize'] = df_prize_result['prize'].replace({
            'Giải Nhất': 'First prize',
            'Giải Nhì': 'Second prize',
            'Giải Ba': 'Third prize'
        })

        # Câu lệnh SQL để chèn dữ liệu vào table power.prize_result_power
        prize_result_query = '''
        MERGE INTO power.prize_result_power AS target
        USING (SELECT	  ? AS prize
                        , ? AS number_of_prizes
                        , ? AS prize_value
                        , ? AS draw_ID
                        , ? AS draw_date
        ) AS source
        ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
        WHEN MATCHED THEN
            UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value

        WHEN NOT MATCHED BY TARGET THEN
            INSERT (prize, number_of_prizes, prize_value, draw_ID, draw_date)
            VALUES (source.prize, source.number_of_prizes, source.prize_value
                    , source.draw_ID, source.draw_date);
        '''
        # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
        for index, row in df_prize_result.iterrows():
            cursor.execute(prize_result_query, row['prize'], row['number_of_prizes'], row['prize_value']
                                            , row['draw_ID'], row['draw_date'])


# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

Đã có đầy đủ dữ liệu power 6/55, không cần crawl mới


## 2. Tải dữ liệu vietlott power 6/55 theo từng draw ID

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

start_draw = int(input('Bạn muốn tải dữ liệu Vietlott Power 6/55 từ kỳ: '))
end_draw = int(input('đến kỳ: '))

drawID_series = [str(i).zfill(5) for i in range(start_draw, end_draw + 1)]

# Vòng lặp để tải dữ liệu về
for drawID in drawID_series:
    # URL của trang web cần crawl
    url = f'https://vietlott.vn/vi/trung-thuong/ket-qua-trung-thuong/655?id={drawID}&nocatche=1'


    # Gửi yêu cầu HTTP GET đến URL
    response = requests.get(url)

    # Sử dụng BeautifulSoup để phân tích cú pháp HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lấy giá trị ngày của dữ liệu
    h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
    date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
    date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

    ######### 1. Tải dữ liệu về số trúng giải #################

    # Tìm kiếm dữ liệu cần thiết
    # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
    draw_data = soup.find_all('div', class_='day_so_ket_qua_v2')

    # Lấy các kết quả trả về dưới dạng text
    for item in draw_data:
        draw = item.text
    # result = [item.text for item in draw_data]
    # Tách chuỗi thành phần số quả bóng và số đặc biệt
    balls, ball_special = draw.split('|')
    # Tách các số quả bóng
    ball_number = [balls[i:i+2] for i in range(1, len(balls), 2)]
    # Tạo dictionary
    draw_result = {'draw_ID': drawID, 'draw_date': date_object}
    draw_result.update({f'ball_{i+1}': ball_number[i] for i in range(len(ball_number))})
    draw_result['ball_special'] = ball_special
    # Tạo DataFrame từ dữ liệu và thêm vào danh sách
    draw_result = {key: [value] for key, value in draw_result.items()}
    df_draw_result = pd.DataFrame(draw_result)
    
    # Câu lệnh SQL để chèn dữ liệu vào table power.draw_result_power
    draw_result_query = '''
    MERGE INTO power.draw_result_power AS target
	USING (SELECT	  ? AS draw_ID
					, ? AS draw_date
					, ? AS ball_1
					, ? AS ball_2
					, ? AS ball_3
					, ? AS ball_4
					, ? AS ball_5
					, ? AS ball_6
					, ? AS ball_special
	) AS source
	ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date
	WHEN MATCHED THEN
		UPDATE SET	target.ball_1 = source.ball_1, target.ball_2 = source.ball_2,
					target.ball_3 = source.ball_3, target.ball_4 = source.ball_4,
					target.ball_5 = source.ball_5, target.ball_6 = source.ball_6,
					target.ball_special = source.ball_special
	WHEN NOT MATCHED BY TARGET THEN
		INSERT (draw_ID, draw_date, ball_1, ball_2, ball_3, ball_4, ball_5, ball_6, ball_special)
		VALUES (source.draw_ID, source.draw_date, source.ball_1, source.ball_2
				, source.ball_3, source.ball_4, source.ball_5, source.ball_6, source.ball_special);
    '''
    # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
    for index, row in df_draw_result.iterrows():
        cursor.execute(draw_result_query, row['draw_ID'], row['draw_date'], row['ball_1'], row['ball_2'], row['ball_3'], row['ball_4'], row['ball_5'], row['ball_6'], row['ball_special'])



    ############################################

    ######### 2. Tải dữ liệu giá trị và số lượng giải #################
    # Tìm kiếm dữ liệu cần thiết
    # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
    prize_data = soup.find('div', class_='table-responsive').find('table')
    # Trích xuất tiêu đề cột
    headers = []
    for th in prize_data.find('thead').find_all('th'):
        headers.append(th.text.strip())
    # Trích xuất dữ liệu hàng
    rows = []
    for tr in prize_data.find('tbody').find_all('tr'):
        cells = tr.find_all('td')
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)
    # Tạo DataFrame
    df_prize_result = pd.DataFrame(rows, columns=headers)
    # Bỏ cột "Kết quả"
    df_prize_result = df_prize_result.drop(columns=["Kết quả"])
    # Đổi tên các cột
    df_prize_result = df_prize_result.rename(columns={
        "Giải thưởng": "prize",
        "Số lượng giải": "number_of_prizes",
        "Giá trị giải (đồng)": "prize_value"
    })
    # Chuyển đổi định dạng của cột number_of_prizes và prize_value
    # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
    df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
    df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')
    df_prize_result['draw_ID'] = drawID
    df_prize_result['draw_date'] = date_object
    # Thay đổi các giá trị cột 'prize'
    df_prize_result['prize'] = df_prize_result['prize'].replace({
        'Giải Nhất': 'First prize',
        'Giải Nhì': 'Second prize',
        'Giải Ba': 'Third prize'
    })

    # Câu lệnh SQL để chèn dữ liệu vào table power.prize_result_power
    prize_result_query = '''
    MERGE INTO power.prize_result_power AS target
	USING (SELECT	  ? AS prize
					, ? AS number_of_prizes
					, ? AS prize_value
					, ? AS draw_ID
					, ? AS draw_date
	) AS source
	ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
	WHEN MATCHED THEN
		UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value

	WHEN NOT MATCHED BY TARGET THEN
		INSERT (prize, number_of_prizes, prize_value, draw_ID, draw_date)
		VALUES (source.prize, source.number_of_prizes, source.prize_value
				, source.draw_ID, source.draw_date);
    '''
    # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
    for index, row in df_prize_result.iterrows():
        cursor.execute(prize_result_query, row['prize'], row['number_of_prizes'], row['prize_value'], row['draw_ID'], row['draw_date'])


# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

## 3. Tải dữ liệu vietlott mega 6/45 tự động theo draw ID chưa được tải

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# Tạo list những draw_ID đã tải về table vietlott.mega.draw_result_mega
cursor.execute('SELECT draw_ID FROM vietlott.mega.draw_result_mega ORDER BY draw_ID')
rows_draw = cursor.fetchall()
draw_result = [row[0] for row in rows_draw]

drawID_draw_result = [str(i).zfill(5) for i in draw_result]

# Tạo list những draw_ID đã tải về table vietlott.mega.prize_result_mega
cursor.execute('SELECT DISTINCT draw_ID FROM vietlott.mega.prize_result_mega ORDER BY draw_ID')
rows_prize = cursor.fetchall()
prize_result = [row[0] for row in rows_prize]

drawID_prize_result = [str(i).zfill(5) for i in prize_result]

# Tạo list chứa những draw_ID đã có từ trước đến hiện tại
winning_url = 'https://vietlott.vn/vi/trung-thuong/ket-qua-trung-thuong/winning-number-645'
response = requests.get(winning_url)
soup = BeautifulSoup(response.text, 'html.parser')
a_tag = soup.find('tbody').find('a')
max_draw_ID = a_tag.text.strip()
max_draw_ID = int(max_draw_ID)

list_drawID = [str(i).zfill(5) for i in range(1, max_draw_ID + 1)]

# Tạo list các drawID còn thiếu và chưa được tải về
missing_drawID = sorted((set(list_drawID) - set(drawID_draw_result)) | (set(list_drawID) - set(drawID_prize_result)))



if not missing_drawID:
    print('Đã có đầy đủ dữ liệu mega 6/45, không cần crawl mới')
else:
    for drawID in missing_drawID:
        # URL của trang web cần crawl
        url = f'https://vietlott.vn/vi/trung-thuong/ket-qua-trung-thuong/645?id={drawID}&nocatche=1'


        # Gửi yêu cầu HTTP GET đến URL
        response = requests.get(url)

        # Sử dụng BeautifulSoup để phân tích cú pháp HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy giá trị ngày của dữ liệu
        h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
        date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
        date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

        ######### 1. Tải dữ liệu về số trúng giải #################

        # Tìm kiếm dữ liệu cần thiết
        # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
        draw_data = soup.find_all('div', class_='day_so_ket_qua_v2')

        # Lấy các kết quả trả về dưới dạng text
        for item in draw_data:
            balls = item.text
        # Tách các số quả bóng
        balls = balls.strip()
        ball_number = [balls[i:i+2] for i in range(0, len(balls), 2)]
        # Tạo dictionary
        draw_result = {'draw_ID': drawID, 'draw_date': date_object}
        draw_result.update({f'ball_{i+1}': ball_number[i] for i in range(len(ball_number))})
        # Tạo DataFrame từ dữ liệu và thêm vào danh sách
        draw_result = {key: [value] for key, value in draw_result.items()}
        df_draw_result = pd.DataFrame(draw_result)
        
        # Câu lệnh SQL để chèn dữ liệu vào table mega.draw_result_mega
        draw_result_query = '''
        MERGE INTO mega.draw_result_mega AS target
        USING (SELECT	  ? AS draw_ID
                        , ? AS draw_date
                        , ? AS ball_1
                        , ? AS ball_2
                        , ? AS ball_3
                        , ? AS ball_4
                        , ? AS ball_5
                        , ? AS ball_6
        ) AS source
        ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date
        WHEN MATCHED THEN
            UPDATE SET	target.ball_1 = source.ball_1, target.ball_2 = source.ball_2,
                        target.ball_3 = source.ball_3, target.ball_4 = source.ball_4,
                        target.ball_5 = source.ball_5, target.ball_6 = source.ball_6
        WHEN NOT MATCHED BY TARGET THEN
            INSERT (draw_ID, draw_date, ball_1, ball_2, ball_3, ball_4, ball_5, ball_6)
            VALUES (source.draw_ID, source.draw_date, source.ball_1, source.ball_2
                    , source.ball_3, source.ball_4, source.ball_5, source.ball_6);
        '''
        # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
        for index, row in df_draw_result.iterrows():
            cursor.execute(draw_result_query, row['draw_ID'], row['draw_date'], row['ball_1'], row['ball_2']
                                            , row['ball_3'], row['ball_4'], row['ball_5'], row['ball_6'])


        ############################################

        ######### 2. Tải dữ liệu giá trị và số lượng giải #################
        # Tìm kiếm dữ liệu cần thiết
        # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
        prize_data = soup.find('div', class_='table-responsive').find('table')
        # Trích xuất tiêu đề cột
        headers = []
        for th in prize_data.find('thead').find_all('th'):
            headers.append(th.text.strip())
        # Trích xuất dữ liệu hàng
        rows = []
        for tr in prize_data.find('tbody').find_all('tr'):
            cells = tr.find_all('td')
            row_data = [cell.text.strip() for cell in cells]
            rows.append(row_data)
        # Tạo DataFrame
        df_prize_result = pd.DataFrame(rows, columns=headers)
        # Bỏ cột "Kết quả"
        df_prize_result = df_prize_result.drop(columns=["Kết quả"])
        # Đổi tên các cột
        df_prize_result = df_prize_result.rename(columns={
            "Giải thưởng": "prize",
            "Số lượng giải": "number_of_prizes",
            "Giá trị giải (đồng)": "prize_value"
        })
        # Chuyển đổi định dạng của cột number_of_prizes và prize_value
        # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
        df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
        df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')
        df_prize_result['draw_ID'] = drawID
        df_prize_result['draw_date'] = date_object
        # Thay đổi các giá trị cột 'prize'
        df_prize_result['prize'] = df_prize_result['prize'].replace({
            'Giải Nhất': 'First prize',
            'Giải Nhì': 'Second prize',
            'Giải Ba': 'Third prize'
        })

        # Câu lệnh SQL để chèn dữ liệu vào table mega.prize_result_mega
        prize_result_query = '''
        MERGE INTO mega.prize_result_mega AS target
        USING (SELECT	  ? AS prize
                        , ? AS number_of_prizes
                        , ? AS prize_value
                        , ? AS draw_ID
                        , ? AS draw_date
        ) AS source
        ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
        WHEN MATCHED THEN
            UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value

        WHEN NOT MATCHED BY TARGET THEN
            INSERT (prize, number_of_prizes, prize_value, draw_ID, draw_date)
            VALUES (source.prize, source.number_of_prizes, source.prize_value
                    , source.draw_ID, source.draw_date);
        '''
        # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
        for index, row in df_prize_result.iterrows():
            cursor.execute(prize_result_query, row['prize'], row['number_of_prizes'], row['prize_value']
                                            , row['draw_ID'], row['draw_date'])
        
        if int(drawID) % 50 == 0:
            print('Đã tải xong dữ liệu số: ', drawID)


# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

Đã có đầy đủ dữ liệu mega 6/45, không cần crawl mới


## 4. Tải dữ liệu vietlott mega 6/55 theo từng draw ID

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

drawID_series = [str(i).zfill(5) for i in range(1126, 1127 + 1)]

for drawID in drawID_series:
    # URL của trang web cần crawl
    url = f'https://vietlott.vn/vi/trung-thuong/ket-qua-trung-thuong/645?id={drawID}&nocatche=1'


    # Gửi yêu cầu HTTP GET đến URL
    response = requests.get(url)

    # Sử dụng BeautifulSoup để phân tích cú pháp HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lấy giá trị ngày của dữ liệu
    h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
    date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
    date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

    ######### 1. Tải dữ liệu về số trúng giải #################

    # Tìm kiếm dữ liệu cần thiết
    # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
    draw_data = soup.find_all('div', class_='day_so_ket_qua_v2')

    # Lấy các kết quả trả về dưới dạng text
    for item in draw_data:
        balls = item.text
    # Tách các số quả bóng
    balls = balls.strip()
    ball_number = [balls[i:i+2] for i in range(0, len(balls), 2)]
    # Tạo dictionary
    draw_result = {'draw_ID': drawID, 'draw_date': date_object}
    draw_result.update({f'ball_{i+1}': ball_number[i] for i in range(len(ball_number))})
    # Tạo DataFrame từ dữ liệu và thêm vào danh sách
    draw_result = {key: [value] for key, value in draw_result.items()}
    df_draw_result = pd.DataFrame(draw_result)
    
    # Câu lệnh SQL để chèn dữ liệu vào table mega.draw_result_mega
    draw_result_query = '''
    MERGE INTO mega.draw_result_mega AS target
    USING (SELECT	  ? AS draw_ID
                    , ? AS draw_date
                    , ? AS ball_1
                    , ? AS ball_2
                    , ? AS ball_3
                    , ? AS ball_4
                    , ? AS ball_5
                    , ? AS ball_6
    ) AS source
    ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date
    WHEN MATCHED THEN
        UPDATE SET	target.ball_1 = source.ball_1, target.ball_2 = source.ball_2,
                    target.ball_3 = source.ball_3, target.ball_4 = source.ball_4,
                    target.ball_5 = source.ball_5, target.ball_6 = source.ball_6
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (draw_ID, draw_date, ball_1, ball_2, ball_3, ball_4, ball_5, ball_6)
        VALUES (source.draw_ID, source.draw_date, source.ball_1, source.ball_2
                , source.ball_3, source.ball_4, source.ball_5, source.ball_6);
    '''
    # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
    for index, row in df_draw_result.iterrows():
        cursor.execute(draw_result_query, row['draw_ID'], row['draw_date'], row['ball_1'], row['ball_2']
                                        , row['ball_3'], row['ball_4'], row['ball_5'], row['ball_6'])


    ############################################

    ######### 2. Tải dữ liệu giá trị và số lượng giải #################
    # Tìm kiếm dữ liệu cần thiết
    # Ví dụ: tìm tất cả các thẻ <div> có class là 'some-class-name'
    prize_data = soup.find('div', class_='table-responsive').find('table')
    # Trích xuất tiêu đề cột
    headers = []
    for th in prize_data.find('thead').find_all('th'):
        headers.append(th.text.strip())
    # Trích xuất dữ liệu hàng
    rows = []
    for tr in prize_data.find('tbody').find_all('tr'):
        cells = tr.find_all('td')
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)
    # Tạo DataFrame
    df_prize_result = pd.DataFrame(rows, columns=headers)
    # Bỏ cột "Kết quả"
    df_prize_result = df_prize_result.drop(columns=["Kết quả"])
    # Đổi tên các cột
    df_prize_result = df_prize_result.rename(columns={
        "Giải thưởng": "prize",
        "Số lượng giải": "number_of_prizes",
        "Giá trị giải (đồng)": "prize_value"
    })
    # Chuyển đổi định dạng của cột number_of_prizes và prize_value
    # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
    df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
    df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')
    df_prize_result['draw_ID'] = drawID
    df_prize_result['draw_date'] = date_object
    # Thay đổi các giá trị cột 'prize'
    df_prize_result['prize'] = df_prize_result['prize'].replace({
        'Giải Nhất': 'First prize',
        'Giải Nhì': 'Second prize',
        'Giải Ba': 'Third prize'
    })

    # Câu lệnh SQL để chèn dữ liệu vào table mega.prize_result_mega
    prize_result_query = '''
    MERGE INTO mega.prize_result_mega AS target
    USING (SELECT	  ? AS prize
                    , ? AS number_of_prizes
                    , ? AS prize_value
                    , ? AS draw_ID
                    , ? AS draw_date
    ) AS source
    ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
    WHEN MATCHED THEN
        UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value

    WHEN NOT MATCHED BY TARGET THEN
        INSERT (prize, number_of_prizes, prize_value, draw_ID, draw_date)
        VALUES (source.prize, source.number_of_prizes, source.prize_value
                , source.draw_ID, source.draw_date);
    '''
    # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
    for index, row in df_prize_result.iterrows():
        cursor.execute(prize_result_query, row['prize'], row['number_of_prizes'], row['prize_value']
                                        , row['draw_ID'], row['draw_date'])


# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

## 5. Tải dữ liệu vietlott max3d pro tự động theo draw ID chưa được tải

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# Tạo list những draw_ID đã tải về table vietlott.mega.prize_result_mega
cursor.execute('SELECT DISTINCT draw_ID FROM vietlott.max3d.prize_result_max3dpro ORDER BY draw_ID')
rows_max3dpro = cursor.fetchall()
max3dpro_result = [row[0] for row in rows_max3dpro]

drawID_max3dpro_result = [str(i).zfill(5) for i in max3dpro_result]

# Tạo list chứa những draw_ID đã có từ trước đến hiện tại
winning_url = 'https://vietlott.vn/en/trung-thuong/ket-qua-trung-thuong/winning-number-max-3Dpro'
response = requests.get(winning_url)
soup = BeautifulSoup(response.text, 'html.parser')
a_tag = soup.find('tbody').find('a')
max_draw_ID = int(a_tag.text.strip())

list_drawID = [str(i).zfill(5) for i in range(1, max_draw_ID + 1)]

# Tạo list các drawID còn thiếu và chưa được tải về
missing_drawID = sorted(set(list_drawID) - set(drawID_max3dpro_result))



if not missing_drawID:
    print('Đã có đầy đủ dữ liệu max3d pro, không cần crawl mới')
else:
    # Lấy các tên cột có trong bảng prize_result_max3dpro
    cursor.execute('''
                SELECT COLUMN_NAME
                FROM INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_CATALOG = 'vietlott'
                AND TABLE_SCHEMA = 'max3d'
                AND TABLE_NAME = 'prize_result_max3dpro'
                ''')
    header = [column[0] for column in cursor.fetchall()]
    del header[-1]

    # Chạy vòng lặp để insert data
    for drawID in missing_drawID:
        # URL của trang web cần crawl
        url = f'https://vietlott.vn/en/trung-thuong/ket-qua-trung-thuong/max-3DPro?id={drawID}&nocatche=1'


        # Gửi yêu cầu HTTP GET đến URL
        response = requests.get(url)

        # Sử dụng BeautifulSoup để phân tích cú pháp HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy giá trị ngày của dữ liệu
        h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
        date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
        date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

        ######### Tải dữ liệu giá trị và số lượng giải #################
        # Tìm kiếm các hàng trong bảng kết quả
        prize_data = soup.select('.chitietketqua_table tr')

        # Tạo danh sách để lưu trữ dữ liệu
        prize_result = []

        # Xử lý từng hàng trong bảng
        for row in prize_data[1:]:  # Bỏ qua hàng đầu tiên vì đó là tiêu đề
            cols = row.find_all('td')
            if cols:
                # Lấy thông tin từ các cột
                prize = cols[0].get_text(strip=True)
                num_of_prizes = cols[2].get_text(strip=True)
                prize_value = cols[3].get_text(strip=True)

                # Lấy kết quả từ các thẻ <span>
                results = [span.get_text(strip=True) for span in cols[1].find_all('span', class_='red')]

                # Thêm các kết quả rỗng nếu cần
                results += [''] * (20 - len(results))

                # Thêm dữ liệu vào danh sách
                prize_result.append([int(drawID), date_object, prize, num_of_prizes, prize_value] + results)

        # Tạo DataFrame
        df_prize_result = pd.DataFrame(prize_result, columns = header) 


        # Chuyển đổi định dạng của cột number_of_prizes và prize_value
        # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
        df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
        df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')

        # Câu lệnh SQL để chèn dữ liệu vào table max3d.prize_result_max3dpro
        prize_result_query = '''
        MERGE INTO max3d.prize_result_max3dpro AS target
        USING (SELECT ? AS draw_ID, ? AS draw_date, ? AS prize, ? AS number_of_prizes, ? AS prize_value
                    , ? AS result_1, ? AS result_2, ? AS result_3, ? AS result_4, ? AS result_5
                    , ? AS result_6, ? AS result_7, ? AS result_8, ? AS result_9, ? AS result_10
                    , ? AS result_11, ? AS result_12, ? AS result_13, ? AS result_14, ? AS result_15
                    , ? AS result_16, ? AS result_17, ? AS result_18, ? AS result_19, ? AS result_20
            ) AS source
        ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
        WHEN MATCHED THEN
            UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value
                        , target.result_1 = source.result_1, target.result_2 = source.result_2, target.result_3 = source.result_3
                        , target.result_4 = source.result_4, target.result_5 = source.result_5, target.result_6 = source.result_6
                        , target.result_7 = source.result_7, target.result_8 = source.result_8, target.result_9 = source.result_9
                        , target.result_10 = source.result_10, target.result_11 = source.result_11, target.result_12 = source.result_12
                        , target.result_13 = source.result_13, target.result_14 = source.result_14, target.result_15 = source.result_15
                        , target.result_16 = source.result_16, target.result_17 = source.result_17, target.result_18 = source.result_18
                        , target.result_19 = source.result_19, target.result_20 = source.result_20
        WHEN NOT MATCHED BY TARGET THEN
            INSERT (draw_ID, draw_date, prize, number_of_prizes, prize_value, result_1, result_2
                    , result_3, result_4, result_5, result_6, result_7, result_8, result_9, result_10, result_11
                    , result_12, result_13, result_14, result_15, result_16, result_17, result_18, result_19, result_20
                    )
            VALUES (source.draw_ID, source.draw_date, source.prize, source.number_of_prizes, source.prize_value
                    , source.result_1, source.result_2, source.result_3, source.result_4, source.result_5
                    , source.result_6, source.result_7, source.result_8, source.result_9, source.result_10
                    , source.result_11, source.result_12, source.result_13, source.result_14, source.result_15
                    , source.result_16, source.result_17, source.result_18, source.result_19, source.result_20
                    );
        '''
        # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
        for index, row in df_prize_result.iterrows():
            cursor.execute(prize_result_query, row['draw_ID'], row['draw_date'], row['prize'], row['number_of_prizes'], row['prize_value']
                        , row['result_1'], row['result_2'], row['result_3'], row['result_4'], row['result_5']
                        , row['result_6'], row['result_7'], row['result_8'], row['result_9'], row['result_10']
                        , row['result_11'], row['result_12'], row['result_13'], row['result_14'], row['result_15']
                        , row['result_16'], row['result_17'], row['result_18'], row['result_19'], row['result_20'])
        if int(drawID) % 10 == 0:
            print('Đã tải xong dữ liệu số: ', int(drawID))

# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

## 6. Tải dữ liệu vietlott max3D pro theo từng draw ID

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# Lấy các tên cột có trong bảng prize_result_max3dpro
cursor.execute('''
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_CATALOG = 'vietlott'
            AND TABLE_SCHEMA = 'max3d'
            AND TABLE_NAME = 'prize_result_max3dpro'
            ''')
header = [column[0] for column in cursor.fetchall()]
del header[-1]

drawID_series = [str(i).zfill(5) for i in range(345, 346 + 1)]
# Chạy vòng lặp để insert data
for drawID in drawID_series:
    # URL của trang web cần crawl
    url = f'https://vietlott.vn/en/trung-thuong/ket-qua-trung-thuong/max-3DPro?id={drawID}&nocatche=1'


    # Gửi yêu cầu HTTP GET đến URL
    response = requests.get(url)

    # Sử dụng BeautifulSoup để phân tích cú pháp HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lấy giá trị ngày của dữ liệu
    h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
    date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
    date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

    ######### Tải dữ liệu giá trị và số lượng giải #################
    # Tìm kiếm các hàng trong bảng kết quả
    prize_data = soup.select('.chitietketqua_table tr')

    # Tạo danh sách để lưu trữ dữ liệu
    prize_result = []

    # Xử lý từng hàng trong bảng
    for row in prize_data[1:]:  # Bỏ qua hàng đầu tiên vì đó là tiêu đề
        cols = row.find_all('td')
        if cols:
            # Lấy thông tin từ các cột
            prize = cols[0].get_text(strip=True)
            num_of_prizes = cols[2].get_text(strip=True)
            prize_value = cols[3].get_text(strip=True)

            # Lấy kết quả từ các thẻ <span>
            results = [span.get_text(strip=True) for span in cols[1].find_all('span', class_='red')]

            # Thêm các kết quả rỗng nếu cần
            results += [''] * (20 - len(results))

            # Thêm dữ liệu vào danh sách
            prize_result.append([int(drawID), date_object, prize, num_of_prizes, prize_value] + results)

    # Tạo DataFrame
    df_prize_result = pd.DataFrame(prize_result, columns = header) 


    # Chuyển đổi định dạng của cột number_of_prizes và prize_value
    # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
    df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
    df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')

    # Câu lệnh SQL để chèn dữ liệu vào table max3d.prize_result_max3dpro
    prize_result_query = '''
    MERGE INTO max3d.prize_result_max3dpro AS target
    USING (SELECT ? AS draw_ID, ? AS draw_date, ? AS prize, ? AS number_of_prizes, ? AS prize_value
                , ? AS result_1, ? AS result_2, ? AS result_3, ? AS result_4, ? AS result_5
                , ? AS result_6, ? AS result_7, ? AS result_8, ? AS result_9, ? AS result_10
                , ? AS result_11, ? AS result_12, ? AS result_13, ? AS result_14, ? AS result_15
                , ? AS result_16, ? AS result_17, ? AS result_18, ? AS result_19, ? AS result_20
        ) AS source
    ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
    WHEN MATCHED THEN
        UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value
                    , target.result_1 = source.result_1, target.result_2 = source.result_2, target.result_3 = source.result_3
                    , target.result_4 = source.result_4, target.result_5 = source.result_5, target.result_6 = source.result_6
                    , target.result_7 = source.result_7, target.result_8 = source.result_8, target.result_9 = source.result_9
                    , target.result_10 = source.result_10, target.result_11 = source.result_11, target.result_12 = source.result_12
                    , target.result_13 = source.result_13, target.result_14 = source.result_14, target.result_15 = source.result_15
                    , target.result_16 = source.result_16, target.result_17 = source.result_17, target.result_18 = source.result_18
                    , target.result_19 = source.result_19, target.result_20 = source.result_20
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (draw_ID, draw_date, prize, number_of_prizes, prize_value, result_1, result_2
                , result_3, result_4, result_5, result_6, result_7, result_8, result_9, result_10, result_11
                , result_12, result_13, result_14, result_15, result_16, result_17, result_18, result_19, result_20
                )
        VALUES (source.draw_ID, source.draw_date, source.prize, source.number_of_prizes, source.prize_value
                , source.result_1, source.result_2, source.result_3, source.result_4, source.result_5
                , source.result_6, source.result_7, source.result_8, source.result_9, source.result_10
                , source.result_11, source.result_12, source.result_13, source.result_14, source.result_15
                , source.result_16, source.result_17, source.result_18, source.result_19, source.result_20
                );
    '''
    # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
    for index, row in df_prize_result.iterrows():
        cursor.execute(prize_result_query, row['draw_ID'], row['draw_date'], row['prize'], row['number_of_prizes'], row['prize_value']
                    , row['result_1'], row['result_2'], row['result_3'], row['result_4'], row['result_5']
                    , row['result_6'], row['result_7'], row['result_8'], row['result_9'], row['result_10']
                    , row['result_11'], row['result_12'], row['result_13'], row['result_14'], row['result_15']
                    , row['result_16'], row['result_17'], row['result_18'], row['result_19'], row['result_20'])
    if int(drawID) % 10 == 0:
        print('Đã tải xong dữ liệu số: ', int(drawID))

# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

## 7. Tải dữ liệu vietlott max3d và max3d+ tự động theo draw ID chưa được tải

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# Tạo list những draw_ID đã tải về table vietlott.max3d.prize_result_max3d
cursor.execute('SELECT draw_ID FROM vietlott.max3d.prize_result_max3d ORDER BY draw_ID')
rows_max3d = cursor.fetchall()
max3d_result = [row[0] for row in rows_max3d]

drawID_max3d_result = [str(i).zfill(5) for i in max3d_result]

# Tạo list những draw_ID đã tải về table vietlott.max3d.prize_result_max3dplus
cursor.execute('SELECT DISTINCT draw_ID FROM vietlott.max3d.prize_result_max3dplus ORDER BY draw_ID')
rows_max3dplus = cursor.fetchall()
max3dplus_result = [row[0] for row in rows_max3dplus]

drawID_max3dplus_result = [str(i).zfill(5) for i in max3dplus_result]

# Tạo list chứa những draw_ID đã có từ trước đến hiện tại
winning_url = 'https://vietlott.vn/en/trung-thuong/ket-qua-trung-thuong/winning-number-max-3D'
response = requests.get(winning_url)
soup = BeautifulSoup(response.text, 'html.parser')
a_tag = soup.find('tbody').find('a')
max_draw_ID = int(a_tag.text.strip())

list_drawID = [str(i).zfill(5) for i in range(1, max_draw_ID + 1)]

# Tạo list các drawID còn thiếu và chưa được tải về
missing_drawID = sorted((set(list_drawID) - set(drawID_max3d_result)) | (set(list_drawID) - set(drawID_max3dplus_result)))



if not missing_drawID:
    print('Đã có đầy đủ dữ liệu max3d pro, không cần crawl mới')
else:
    # Lấy các tên cột có trong bảng prize_result_max3dplus
    cursor.execute('''
                SELECT COLUMN_NAME
                FROM INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_CATALOG = 'vietlott'
                AND TABLE_SCHEMA = 'max3d'
                AND TABLE_NAME = 'prize_result_max3dplus'
                ''')
    header_max3dplus = [column[0] for column in cursor.fetchall()]
    del header_max3dplus[-1]

    # Chạy vòng lặp để insert data
    for drawID in missing_drawID:
        # URL của trang web cần crawl
        url = f'https://vietlott.vn/en/trung-thuong/ket-qua-trung-thuong/max-3D?id={drawID}&nocatche=1'


        # Gửi yêu cầu HTTP GET đến URL
        response = requests.get(url)

        # Sử dụng BeautifulSoup để phân tích cú pháp HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy giá trị ngày của dữ liệu
        h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
        date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
        date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

        ######### Tải dữ liệu giá trị và số lượng giải #################
        # Tìm kiếm các hàng trong bảng kết quả
        prize_data = soup.select('.chitietketqua_table tr')

        # Tạo danh sách để lưu trữ dữ liệu
        prize_result = []

        # Xử lý từng hàng trong bảng
        for row in prize_data[1:]:  # Bỏ qua hàng đầu tiên vì đó là tiêu đề
            cols = row.find_all('td')
            if cols:
                # Lấy thông tin từ các cột
                prize = cols[0].get_text(strip=True)
                num_of_prizes = cols[2].get_text(strip=True)
                prize_value = cols[3].get_text(strip=True)

                # Lấy tất cả các số từ cột kết quả
                if cols[1].find('span'):
                    # Lấy số từ thẻ <span>
                    results = [span.get_text(strip=True) for span in cols[1].find_all('span')]
                else:
                    # Nếu không có thẻ <span>, tách các số từ thẻ <td>
                    results = cols[1].get_text(strip=True).split()

                # Thêm các kết quả rỗng nếu cần
                results += [''] * (20 - len(results))

                # Thêm dữ liệu vào danh sách
                prize_result.append([int(drawID), date_object, prize, num_of_prizes, prize_value] + results)

        # Tạo DataFrame
        df_prize_result = pd.DataFrame(prize_result, columns = header_max3dplus) 

        # Chuyển đổi định dạng của cột number_of_prizes và prize_value
        # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
        df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
        df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')

        # Tạo ra 2 data frame mới tương ứng với 2 bảng max3d và max3dpro
        df_max3d_result = df_prize_result.iloc[:4].drop(columns=[f'result_{i}' for i in range(9, 21)])
        df_max3dplus_result = df_prize_result.iloc[4:]
        df_max3dplus_result.loc[df_max3dplus_result['prize'] == 'Special Prize', ['result_3', 'result_4', 'result_5']] = ''

        # Câu lệnh SQL để chèn dữ liệu vào table max3d.prize_result_max3d
        max3d_result_query = '''
        MERGE INTO max3d.prize_result_max3d AS target
        USING (SELECT ? AS draw_ID, ? AS draw_date, ? AS prize, ? AS number_of_prizes, ? AS prize_value
                    , ? AS result_1, ? AS result_2, ? AS result_3, ? AS result_4, ? AS result_5
                    , ? AS result_6, ? AS result_7, ? AS result_8
            ) AS source
        ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
        WHEN MATCHED THEN
            UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value
                        , target.result_1 = source.result_1, target.result_2 = source.result_2, target.result_3 = source.result_3
                        , target.result_4 = source.result_4, target.result_5 = source.result_5, target.result_6 = source.result_6
                        , target.result_7 = source.result_7, target.result_8 = source.result_8
        WHEN NOT MATCHED BY TARGET THEN
            INSERT (draw_ID, draw_date, prize, number_of_prizes, prize_value, result_1, result_2
                    , result_3, result_4, result_5, result_6, result_7, result_8
                    )
            VALUES (source.draw_ID, source.draw_date, source.prize, source.number_of_prizes, source.prize_value
                    , source.result_1, source.result_2, source.result_3, source.result_4, source.result_5
                    , source.result_6, source.result_7, source.result_8
                    );
        '''
        # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
        for index, row in df_max3d_result.iterrows():
            cursor.execute(max3d_result_query, row['draw_ID'], row['draw_date'], row['prize'], row['number_of_prizes'], row['prize_value']
                        , row['result_1'], row['result_2'], row['result_3'], row['result_4'], row['result_5']
                        , row['result_6'], row['result_7'], row['result_8'])

        # Câu lệnh SQL để chèn dữ liệu vào table max3d.prize_result_max3dplus
        max3dplus_result_query = '''
        MERGE INTO max3d.prize_result_max3dplus AS target
        USING (SELECT ? AS draw_ID, ? AS draw_date, ? AS prize, ? AS number_of_prizes, ? AS prize_value
                    , ? AS result_1, ? AS result_2, ? AS result_3, ? AS result_4, ? AS result_5
                    , ? AS result_6, ? AS result_7, ? AS result_8, ? AS result_9, ? AS result_10
                    , ? AS result_11, ? AS result_12, ? AS result_13, ? AS result_14, ? AS result_15
                    , ? AS result_16, ? AS result_17, ? AS result_18, ? AS result_19, ? AS result_20
            ) AS source
        ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
        WHEN MATCHED THEN
            UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value
                        , target.result_1 = source.result_1, target.result_2 = source.result_2, target.result_3 = source.result_3
                        , target.result_4 = source.result_4, target.result_5 = source.result_5, target.result_6 = source.result_6
                        , target.result_7 = source.result_7, target.result_8 = source.result_8, target.result_9 = source.result_9
                        , target.result_10 = source.result_10, target.result_11 = source.result_11, target.result_12 = source.result_12
                        , target.result_13 = source.result_13, target.result_14 = source.result_14, target.result_15 = source.result_15
                        , target.result_16 = source.result_16, target.result_17 = source.result_17, target.result_18 = source.result_18
                        , target.result_19 = source.result_19, target.result_20 = source.result_20
        WHEN NOT MATCHED BY TARGET THEN
            INSERT (draw_ID, draw_date, prize, number_of_prizes, prize_value, result_1, result_2
                    , result_3, result_4, result_5, result_6, result_7, result_8, result_9, result_10, result_11
                    , result_12, result_13, result_14, result_15, result_16, result_17, result_18, result_19, result_20
                    )
            VALUES (source.draw_ID, source.draw_date, source.prize, source.number_of_prizes, source.prize_value
                    , source.result_1, source.result_2, source.result_3, source.result_4, source.result_5
                    , source.result_6, source.result_7, source.result_8, source.result_9, source.result_10
                    , source.result_11, source.result_12, source.result_13, source.result_14, source.result_15
                    , source.result_16, source.result_17, source.result_18, source.result_19, source.result_20
                    );
        '''
        # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
        for index, row in df_max3dplus_result.iterrows():
            cursor.execute(max3dplus_result_query, row['draw_ID'], row['draw_date'], row['prize'], row['number_of_prizes'], row['prize_value']
                        , row['result_1'], row['result_2'], row['result_3'], row['result_4'], row['result_5']
                        , row['result_6'], row['result_7'], row['result_8'], row['result_9'], row['result_10']
                        , row['result_11'], row['result_12'], row['result_13'], row['result_14'], row['result_15']
                        , row['result_16'], row['result_17'], row['result_18'], row['result_19'], row['result_20'])        
        
        if int(drawID) % 50 == 0:
            print('Đã tải xong dữ liệu số: ', int(drawID))

# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()

## 8. Tải dữ liệu vietlott max3D và max3D plus theo từng draw ID

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pyodbc

# Kết nối đến SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=WHITEKING\WHITEKING;'
                      'DATABASE=vietlott;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# Lấy các tên cột có trong bảng prize_result_max3dplus
cursor.execute('''
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_CATALOG = 'vietlott'
            AND TABLE_SCHEMA = 'max3d'
            AND TABLE_NAME = 'prize_result_max3dplus'
            ''')
header_max3dplus = [column[0] for column in cursor.fetchall()]
del header_max3dplus[-1]

drawID_series = [str(i).zfill(5) for i in range(698, 699 + 1)]
# Chạy vòng lặp để insert data
for drawID in drawID_series:
    # URL của trang web cần crawl
    url = f'https://vietlott.vn/en/trung-thuong/ket-qua-trung-thuong/max-3D?id={drawID}&nocatche=1'


    # Gửi yêu cầu HTTP GET đến URL
    response = requests.get(url)

    # Sử dụng BeautifulSoup để phân tích cú pháp HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lấy giá trị ngày của dữ liệu
    h5_tag = soup.find('div', class_='chitietketqua_title').find('h5')
    date_str = h5_tag.find_all('b')[-1].text  # Lấy nội dung của thẻ <b> cuối cùng
    date_object = datetime.strptime(date_str, '%d/%m/%Y').date()

    ######### Tải dữ liệu giá trị và số lượng giải #################
    # Tìm kiếm các hàng trong bảng kết quả
    prize_data = soup.select('.chitietketqua_table tr')

    # Tạo danh sách để lưu trữ dữ liệu
    prize_result = []

    # Xử lý từng hàng trong bảng
    for row in prize_data[1:]:  # Bỏ qua hàng đầu tiên vì đó là tiêu đề
        cols = row.find_all('td')
        if cols:
            # Lấy thông tin từ các cột
            prize = cols[0].get_text(strip=True)
            num_of_prizes = cols[2].get_text(strip=True)
            prize_value = cols[3].get_text(strip=True)

            # Lấy tất cả các số từ cột kết quả
            if cols[1].find('span'):
                # Lấy số từ thẻ <span>
                results = [span.get_text(strip=True) for span in cols[1].find_all('span')]
            else:
                # Nếu không có thẻ <span>, tách các số từ thẻ <td>
                results = cols[1].get_text(strip=True).split()

            # Thêm các kết quả rỗng nếu cần
            results += [''] * (20 - len(results))

            # Thêm dữ liệu vào danh sách
            prize_result.append([int(drawID), date_object, prize, num_of_prizes, prize_value] + results)

    # Tạo DataFrame
    df_prize_result = pd.DataFrame(prize_result, columns = header_max3dplus) 

    # Chuyển đổi định dạng của cột number_of_prizes và prize_value
    # Loại bỏ các dấu phân cách (ví dụ: dấu chấm đề phân cách hàng nghìn)
    df_prize_result["number_of_prizes"] = df_prize_result["number_of_prizes"].str.replace(".", "").astype(int)
    df_prize_result["prize_value"] = df_prize_result["prize_value"].str.replace(".", "").str.replace("đồng", "").astype('int64')

    # Tạo ra 2 data frame mới tương ứng với 2 bảng max3d và max3dpro
    df_max3d_result = df_prize_result.iloc[:4].drop(columns=[f'result_{i}' for i in range(9, 21)])
    df_max3dplus_result = df_prize_result.iloc[4:]
    df_max3dplus_result.loc[df_max3dplus_result['prize'] == 'Special Prize', ['result_3', 'result_4', 'result_5']] = ''

    # Câu lệnh SQL để chèn dữ liệu vào table max3d.prize_result_max3d
    max3d_result_query = '''
    MERGE INTO max3d.prize_result_max3d AS target
    USING (SELECT ? AS draw_ID, ? AS draw_date, ? AS prize, ? AS number_of_prizes, ? AS prize_value
                , ? AS result_1, ? AS result_2, ? AS result_3, ? AS result_4, ? AS result_5
                , ? AS result_6, ? AS result_7, ? AS result_8
        ) AS source
    ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
    WHEN MATCHED THEN
        UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value
                    , target.result_1 = source.result_1, target.result_2 = source.result_2, target.result_3 = source.result_3
                    , target.result_4 = source.result_4, target.result_5 = source.result_5, target.result_6 = source.result_6
                    , target.result_7 = source.result_7, target.result_8 = source.result_8
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (draw_ID, draw_date, prize, number_of_prizes, prize_value, result_1, result_2
                , result_3, result_4, result_5, result_6, result_7, result_8
                )
        VALUES (source.draw_ID, source.draw_date, source.prize, source.number_of_prizes, source.prize_value
                , source.result_1, source.result_2, source.result_3, source.result_4, source.result_5
                , source.result_6, source.result_7, source.result_8
                );
    '''
    # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
    for index, row in df_max3d_result.iterrows():
        cursor.execute(max3d_result_query, row['draw_ID'], row['draw_date'], row['prize'], row['number_of_prizes'], row['prize_value']
                    , row['result_1'], row['result_2'], row['result_3'], row['result_4'], row['result_5']
                    , row['result_6'], row['result_7'], row['result_8'])

    # Câu lệnh SQL để chèn dữ liệu vào table max3d.prize_result_max3dplus
    max3dplus_result_query = '''
    MERGE INTO max3d.prize_result_max3dplus AS target
    USING (SELECT ? AS draw_ID, ? AS draw_date, ? AS prize, ? AS number_of_prizes, ? AS prize_value
                , ? AS result_1, ? AS result_2, ? AS result_3, ? AS result_4, ? AS result_5
                , ? AS result_6, ? AS result_7, ? AS result_8, ? AS result_9, ? AS result_10
                , ? AS result_11, ? AS result_12, ? AS result_13, ? AS result_14, ? AS result_15
                , ? AS result_16, ? AS result_17, ? AS result_18, ? AS result_19, ? AS result_20
        ) AS source
    ON target.draw_ID = source.draw_ID AND target.draw_date = source.draw_date AND target.prize = source.prize
    WHEN MATCHED THEN
        UPDATE SET	target.number_of_prizes = source.number_of_prizes, target.prize_value = source.prize_value
                    , target.result_1 = source.result_1, target.result_2 = source.result_2, target.result_3 = source.result_3
                    , target.result_4 = source.result_4, target.result_5 = source.result_5, target.result_6 = source.result_6
                    , target.result_7 = source.result_7, target.result_8 = source.result_8, target.result_9 = source.result_9
                    , target.result_10 = source.result_10, target.result_11 = source.result_11, target.result_12 = source.result_12
                    , target.result_13 = source.result_13, target.result_14 = source.result_14, target.result_15 = source.result_15
                    , target.result_16 = source.result_16, target.result_17 = source.result_17, target.result_18 = source.result_18
                    , target.result_19 = source.result_19, target.result_20 = source.result_20
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (draw_ID, draw_date, prize, number_of_prizes, prize_value, result_1, result_2
                , result_3, result_4, result_5, result_6, result_7, result_8, result_9, result_10, result_11
                , result_12, result_13, result_14, result_15, result_16, result_17, result_18, result_19, result_20
                )
        VALUES (source.draw_ID, source.draw_date, source.prize, source.number_of_prizes, source.prize_value
                , source.result_1, source.result_2, source.result_3, source.result_4, source.result_5
                , source.result_6, source.result_7, source.result_8, source.result_9, source.result_10
                , source.result_11, source.result_12, source.result_13, source.result_14, source.result_15
                , source.result_16, source.result_17, source.result_18, source.result_19, source.result_20
                );
    '''
    # Duyệt qua mỗi hàng trong DataFrame và insert vào SQL Server
    for index, row in df_max3dplus_result.iterrows():
        cursor.execute(max3dplus_result_query, row['draw_ID'], row['draw_date'], row['prize'], row['number_of_prizes'], row['prize_value']
                    , row['result_1'], row['result_2'], row['result_3'], row['result_4'], row['result_5']
                    , row['result_6'], row['result_7'], row['result_8'], row['result_9'], row['result_10']
                    , row['result_11'], row['result_12'], row['result_13'], row['result_14'], row['result_15']
                    , row['result_16'], row['result_17'], row['result_18'], row['result_19'], row['result_20'])        
    
    if int(drawID) % 50 == 0:
        print('Đã tải xong dữ liệu số: ', int(drawID))

# Cam kết giao dịch
conn.commit()

# Đóng kết nối
cursor.close()
conn.close()